# Python Intro - Final Exercise


In [144]:
import pyodbc
import pandas as pd
import numpy as np
print('ok')

ok


In [145]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [146]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 
courses = pd.read_sql("SELECT * FROM Courses;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn) 
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

#add another column for join with department table
courses['DepartmentId'] = courses.DepartmentID

data = pd.merge(pd.merge(pd.merge(pd.merge(classrooms, courses, on='CourseId'), 
                                    departments, on='DepartmentId'), 
                           students, on='StudentId'), 
                  teachers, on='TeacherId')

data = data.rename(columns = {'FirstName_x':'StudentFirstName', 
                         'LastName_x': 'StudentLastName',
                        'Gender_x' : 'StudentGender',
                        'FirstName_y':'TeacherFirstName',
                        'LastName_y':'TeaccherLastName',
                        'Gender_y':'TeacherGender'})



# Questions
 
 
## Q1. Count the number of students on each department

In [147]:
print('\nNumber of students : %d' % students.shape[0])


Number of students : 280


## Q2. How many students have each course of the English department and the total number of students in the department?

In [178]:
#total student in each department
res = data[['DepartmentName', 'StudentId']].drop_duplicates().groupby('DepartmentName').agg({'StudentId':['count']})

print("Number of students in all departments is\n", res)


#in english department
res1 = data[['DepartmentName', 'StudentId']].drop_duplicates();
print("Number of students in Engllish depsrtment is", res1[res1.DepartmentName == 'English'].shape[0])


data['CourseNum'] = 0
#tt = data[data.DepartmentName == 'English'].groupby('StudentId').CourseNum.agg('count').reset_index()

tt = data[data.DepartmentName == 'English'].groupby('StudentId')['CourseNum'].agg('count')



print ("Number of student that took all the 3 english classes is:", tt[tt==3].shape[0])

Number of students in all departments is
                StudentId
                   count
DepartmentName          
Arts                 137
English               84
Science              200
Sport                 81
Number of students in Engllish depsrtment is 84
Number of student that took all the 3 english classes is: 0


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [180]:
data['Res'] = -1 
science_classes = data[data.DepartmentName == 'Science'].groupby('CourseId').Res.agg('count')

print("Number of small classrooms needed for Science Department courses is: ",science_classes[science_classes < 22].shape[0])

print("Number of large classrooms needed for Science Department courses is: ",science_classes[science_classes >= 22].shape[0])



Number of small classrooms needed for Science Department courses is:  6
Number of large classrooms needed for Science Department courses is:  10


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [185]:
data[['StudentId', 'StudentGender']].drop_duplicates().groupby('StudentGender').agg('count')

,StudentId
StudentGender,
F,153
M,103


## Q5. For which courses the percentage of male/female students is over 70%?

In [228]:
data.head()

data['Female'] = 0
data['Male'] = 0


data.loc[data.StudentGender == 'F','Female'] = 1
data.loc[data.StudentGender == 'M','Male'] = 1

df = data.groupby('CourseName').agg({'Female':['mean']})
if (df[df > 0.7].dropna().shape[0] > 0 ):
    print("Courses with females more than 70%: \n", df[df > 0.7].dropna())

df = data.groupby('CourseName').agg({'Male':['mean']})
if (df[df > 0.7].dropna().shape[0] > 0 ):
    print("Courses with males more than 70%: \n", df[df > 0.7].dropna())




Courses with females more than 70%:                Female
                mean
CourseName          
Sculpture   0.708333
Tenis       0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [241]:
data.head()

df = data[['DepartmentName', 'StudentId', 'degree']]

df = df[df['degree'] > 80].drop_duplicates()

print("For each department number of students passed with a grade over 80\n", df.groupby('DepartmentName')['StudentId'].agg('count'))

For each department number of students passed with a grade over 80
 DepartmentName
Arts        68
English     43
Science    171
Sport       41
Name: StudentId, dtype: int64


## Q7. For each department, how many students passed with a grades under 60?

In [239]:
data.head()

df = data[['DepartmentName', 'StudentId', 'degree']]

df = df[df['degree'] < 60].drop_duplicates()

print("For each department number of students passed with a grade under 60\n", df.groupby('DepartmentName')['StudentId'].agg('count'))

For each department number of students passed with a grade under 60
 DepartmentName
Arts       3
English    2
Science    7
Sport      1
Name: StudentId, dtype: int64


## Q8. Rate the teachers by their average student's grades (in descending order).

In [251]:
data.head()

print(data.groupby(['TeacherFirstName','TeaccherLastName','TeacherId'])['degree'].agg('mean').sort_values(ascending=False))


TeacherFirstName  TeaccherLastName  TeacherId
 Darick           Hess              22.0         81.432703
 Jordyn           Baron             3.0          80.729494
 Heather          Metcalf           5.0          80.440242
 Zachary          Healy             20.0         79.365867
 Kaylea           Monzingo          1.0          79.365537
 Daniel           Bland             18.0         78.908281
 Keegan           Price             24.0         78.814631
 Conor            Mckittrick        15.0         78.699492
 Maxwell          Romkee            23.0         78.673509
 Terran           Beaty             19.0         78.495479
 Tanner           Fitzsimons        17.0         78.443395
 Mackenzie        Olsen             10.0         78.383348
 Ernest           Stoyanov          21.0         78.316150
 Madison          Batan             6.0          77.743575
 Katerina         Young             9.0          77.406852
 Mikayla          Russell           12.0         77.134323
 Amanda   

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).